In [ ]:
import roop.globals
import roop.ui as ui
import shutil
from roop.processors.frame.core import get_frame_processors_modules
from roop.utilities import normalize_output_path
from utils import *

In [ ]:
upsampler, codeformer_net, device = get_codeformer_models()

In [ ]:
def encode_image(image: Image, img_format: int = "JPEG") -> str:
    """
    Encodes image to base64 string.
    Args:
        image: PIL image
        img_format: image format
    Returns:
        base64 string
    """
    img_byte_arr = io.BytesIO()
    image.save(img_byte_arr, format=img_format)
    img_byte_arr = img_byte_arr.getvalue()
    img_byte_arr = base64.b64encode(img_byte_arr).decode("utf-8")
    return img_byte_arr


def decode_image(image: str) -> Image:
    """
    Decodes base64 string to PIL image.
    Args:
        image: base64 string
    Returns:
        PIL image
    """
    img_byte_arr = base64.b64decode(image)
    img_byte_arr = io.BytesIO(img_byte_arr)
    img_byte_arr = Image.open(img_byte_arr)
    return img_byte_arr


def image_grid(imgs: List[Any], rows: int = 2, cols: int = 2):
    """
    Creates a grid of images.
    Args:
        imgs: list of PIL images
        rows: number of rows
        cols: number of columns
    Returns:
        PIL image
    """
    w, h = imgs[0].size
    grid = Image.new("RGB", size=(cols * w, rows * h))

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i % cols * w, i // cols * h))
    return grid

In [ ]:
input_folder = "/home/azureuser/matt_code/roop_pons/inputs"
bases_folder = "/home/azureuser/matt_code/roop_pons/bases"
input_imgs = [f"{input_folder}/{img}" for img in os.listdir(input_folder)]
base_imgs = [f"{bases_folder}/{img}" for img in os.listdir(bases_folder)]
input_imgs.sort()
base_imgs.sort()

In [ ]:
from roop.utilities import normalize_output_path

roop.globals.many_faces = False
roop.globals.reference_face_position = 0
roop.globals.reference_frame_number = 0
roop.globals.similar_face_distance = 0.85
roop.globals.execution_providers = decode_execution_providers(["cpu"])
roop.globals.execution_threads = suggest_execution_threads()
roop.globals.headless = True

In [ ]:
frame_processors = ["face_swapper"]
for frame_processor in get_frame_processors_modules(frame_processors):
    frame_processor.pre_check()

In [ ]:
import cv2
import concurrent.futures

imgs = []
args_roop = []
for i in range(4):
    in_img = Image.open(input_imgs[i])
    base_img = Image.open(base_imgs[i])
    arg_roop = (in_img, base_img)
    args_roop.append(arg_roop)

frame_processors = ["face_swapper"]
# shutil.copy2(base_img, roop.globals.output_path)
for frame_processor in get_frame_processors_modules(frame_processors):
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        results = list(executor.map(frame_processor.process_image, args_roop))

imgs = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in results]

args_codeformer = [
    (img, True, False, False, 0, 1.0, upsampler, codeformer_net, device) for img in imgs
]
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(inference_codeformer, args_codeformer))
imgs = [Image.fromarray(img) for img in results]
grid = image_grid(imgs, cols=len(imgs) // 2, rows=2)
plt.figure(figsize=(70, 10))
plt.imshow(grid)
plt.show()